## **Extracción de Características Visuales (Visual Embeddings)**

En este notebook abordamos la fase de **Feature Extraction** (Extracción de Características) para la modalidad de vídeo. El objetivo principal es transformar la información "cruda" de los vídeos (píxeles) en representaciones compactas de significado (**Embeddings**), que posteriormente alimentarán a los modelos de Deep Learning en la fase de experimentación.

Este paso es esencial ya que los vídeos originales de ambos *datasets* contienen una gran cantidad de información redundante (miles de píxeles por frame, repetidos 30 veces por segundo, por ejemplo). Entrenar nuestros modelos directamente con los píxeles en crudo es computacionalmente inviable y puede derivar al *overfitting* de nuestros. Por tanto, aplicamos una estrategia de **Transfer Learning** donde utilizaremos redes neuronales convolucionales (CNNs) y transformers ya pre-entrenadas en millones de imágenes (ImageNet) para que actúen como "extractores de características". Estas redes ya *saben ver* (detectan bordes, formas, texturas y patrones faciales complejos), por lo que podemos usar sus capas internas para resumir cada frame/s del vídeo en un vector numérico de alta densidad.

Para ello, siguiendo la metodología, extraemos las características de **tres arquitecturas diferentes seleccionadas** para poder comparar su rendimiento:

1. **ResNet50 (Residual Networks):** Estándar en la industria de la Visión Artificial. Utiliza conexiones residuales para permitir redes muy profundas. Genera vectores de **2048 dimensiones** por defecto.

2. **EfficientNet-B0:** Arquitectura diseñada para ser extremadamente eficiente en recursos computacionales sin perder precisión. Genera vectores de **1280 dimensiones** por defecto, lo que ahorrará espacio y tiempo de cómputo.

3. **ViT-B/16 (Vision Transformer):** Arquitectura base basada en Transformer que ha demostrado excelente rendimiento en tareas de visión artificial. Procesa las imágenes como secuencias de parches y genera vectores de **768 dimensiones** por defecto. Se ha seleccionado la versión **Base** de **ViT** en lugar del **Large** (1024) ya que, como ya hemos justificado, los vídeos de MELD e IEMOCAP son vídeos cortos en su mayoría, por tanto una dimensión de 1024 introduciría demasiada redundancia, lo que provocaría que la red hiciera *overfitting*.


**NOTA:** Dado que la emoción es dinámica, no podemos usar solo una imagen, pero tampoco el vídeo entero. Se lleva a cabo a continuación un **análisis previo** para la decisión del número de *frames* que extraeremos del vídeo.

In [ ]:
# Carga previa de todas las librerías y paquetes necesarios
import os
import cv2 # Para leer los vídeos
import numpy as np  
import torch 
import pandas as pd
import torch.nn as nn  # Para modificar las capas finales de los modelos (eliminarlas) 
from torchvision import models, transforms 
from PIL import Image
from tqdm import tqdm  # Para barra de progreso
import shutil
from google.colab import drive
import zipfile
import tarfile

# Configuración de GPU/CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Dispositivo: {device}")

Dispositivo: cuda


In [3]:
# Se monta Drive para acceder a los datos y guardar los resultados:
drive.mount('/content/drive')

Mounted at /content/drive


Definimos las rutas desde las cuales se cargan los archivos comprimidos con los *embeddings* desde **Drive**.

In [ ]:
LOCAL_DATA_ROOT = '/content/data' # Directorio local en Colab para almacenar los datos

# ----- RUTAS ORIGEN (DRIVE) ----
# Archivos .tar.gz con los embeddings ya extraídos
TAR_PATH_RESNET = '/content/drive/MyDrive/Proyecto_TFG_Data/EMBEDDINGS_VISUAL/features_resnet.tar.gz'  
TAR_PATH_EFFICIENTNET = '/content/drive/MyDrive/Proyecto_TFG_Data/EMBEDDINGS_VISUAL/features_efficientnet.tar.gz'
TAR_PATH_VIT = '/content/drive/MyDrive/Proyecto_TFG_Data/EMBEDDINGS_VISUAL/features_vit.tar.gz'

MELD_CLIPS_PATH = os.path.join(LOCAL_DATA_ROOT, 'MELD_CLIPS')  # Directorio donde se descomprimirán los vídeos de MELD
IEMOCAP_CLIPS_PATH = os.path.join(LOCAL_DATA_ROOT, 'IEMOCAP_CLIPS')  # Directorio donde se descomprimirán los vídeos de IEMOCAP

# ---------------- RUTAS DESTINO (LOCAL COLAB) ----------------
# Rutas donde se guardarán los embeddings extraídos por cada modelo (después de descomprimir los .tar.gz)
OUTPUT_DIR_RESNET = os.path.join(LOCAL_DATA_ROOT, 'EMBEDDINGS_VISUAL_RESNET')
OUTPUT_DIR_EFFICIENTNET = os.path.join(LOCAL_DATA_ROOT, 'EMBEDDINGS_VISUAL_EFFICIENTNET')
OUTPUT_DIR_VIT = os.path.join(LOCAL_DATA_ROOT, 'EMBEDDINGS_VISUAL_ViT')

MELD_CLIPS_PATH = os.path.join(LOCAL_DATA_ROOT, 'MELD_CLIPS')  
IEMOCAP_CLIPS_PATH = os.path.join(LOCAL_DATA_ROOT, 'IEMOCAP_CLIPS')

# CSV:
DATA_ROOT_CSV = '/content/drive/MyDrive/Proyecto_TFG_Data/Multimodal_Stress_Dataset.csv' 

# ------------ EXTRACCIÓN DE VÍDEOS (.zip) ------------
ZIP_PATH_MELD_CLIPS_DRIVE = '/content/drive/MyDrive/Proyecto_TFG_Data/MELD_CLIPS.zip'  
ZIP_PATH_IEMOCAP_CLIPS_DRIVE = '/content/drive/MyDrive/Proyecto_TFG_Data/IEMOCAP_CLIPS.zip'

zip_files = {
    ZIP_PATH_MELD_CLIPS_DRIVE: MELD_CLIPS_PATH,
    ZIP_PATH_IEMOCAP_CLIPS_DRIVE: IEMOCAP_CLIPS_PATH
}

for zip_drive_path, local_path in zip_files.items():
    if not os.path.exists(local_path) or len(os.listdir(local_path)) == 0:
        print(f"Copiando {zip_drive_path} a {LOCAL_DATA_ROOT}.")
        os.makedirs(LOCAL_DATA_ROOT, exist_ok=True)
        shutil.copy(zip_drive_path, LOCAL_DATA_ROOT)
        with zipfile.ZipFile(os.path.join(LOCAL_DATA_ROOT, os.path.basename(zip_drive_path)), 'r') as zip_ref:
            zip_ref.extractall(LOCAL_DATA_ROOT)
    else:
        print(f"La estructura de directorios de {os.path.basename(zip_drive_path)} ya existe en {LOCAL_DATA_ROOT}. No se realizará la copia ni descompresión.")

Copiando /content/drive/MyDrive/Proyecto_TFG_Data/MELD_CLIPS.zip a /content/data.
Copiando /content/drive/MyDrive/Proyecto_TFG_Data/IEMOCAP_CLIPS.zip a /content/data.


In [7]:
# Carga del dataset global

file_path = DATA_ROOT_CSV
if os.path.exists(file_path):
    df_global = pd.read_csv(file_path)
    display(df_global.head()) 
else:
    print(f"No se encuentra el archivo en {file_path}")

,Utterance_ID,Dialogue_ID,video_path,audio_path,Transcription,duration,split,target_stress,dataset_origin
0,train_dia0_utt0,0,train_splits/dia0_utt0.mp4,MELD_Audio/train_dia0_utt0.wav,also I was the point person on my company's tr...,5.672333,train,0,MELD
1,train_dia0_utt1,0,train_splits/dia0_utt1.mp4,MELD_Audio/train_dia0_utt1.wav,You must've had your hands full.,1.501500,train,0,MELD
2,train_dia0_utt2,0,train_splits/dia0_utt2.mp4,MELD_Audio/train_dia0_utt2.wav,That I did. That I did.,2.919583,train,0,MELD
3,train_dia0_utt3,0,train_splits/dia0_utt3.mp4,MELD_Audio/train_dia0_utt3.wav,So let's talk a little bit about your duties.,2.752750,train,0,MELD
4,train_dia0_utt4,0,train_splits/dia0_utt4.mp4,MELD_Audio/train_dia0_utt4.wav,My duties? All right.,6.464792,train,0,MELD


-----

#### **ESTRUCTURA NOTEBOOK Y EJECUCIÓN**:

Debido a la alta carga computacional que supone el procesado de los *clips* de vídeo de ambos datasets para la extracción de los embeddings, y a esto se le suman los **3 diferentes codificadores** que utilizaremos para ello, se ha optado por realizar la carga y ejecución en el Servidor NVIDIA DGX de **CyberDataLab**, al cual he tenido acceso para realizar el TFG. 

A continuación, se presenta el análisis inicial llevado a cabo, previo a la extracción de características visuales, y, con un fin demostrativo, se indica y explica el código de inicialización de cada modelo a utilizar (**ResNet50**, **EfficientNet-B0** y **ViT-B/16**), así como el *script* final lanzado en el servidor para la ejecución. 

Finalmente, se cargan los *embeddings* resultantes (empaquetados y almacenados en **Drive**) en el servidor local de **Colab** para realizar una auditoría final de validación (**sanity check**).


---

### **Análisis Previo**

* Identificamos el número de *frames* promedio en una muestra aleatoria para justificar la decisión de cuántos *frames* se extraerán del video:

In [ ]:
# --- ANÁLISIS PREVIO DE FRAMES ---

def analizar_densidad_frames(df,path_clips_meld, path_clips_iemocap, sample_size=50):
    """
    Selecciona un batch aleatorio POR CADA DATASET (MELD e IEMOCAP) de vídeos de acuerdo a la longitud indicada (sample_size).
    Devuelve:
    - resultados: Lista que incluye los resultados de la media (mean), mínimo (min) y máximo (max) de los vídeos del bacth por cada dataset (MELD e IEMOCAP)
    """
    resultados = []  # Aquí almacenamos los resultados obtenidos a devolver
    for origen in ['MELD','IEMOCAP'] :
        subset = df[df['dataset_origin'] == origen]  # Seleccionamos vídeos de dicho dataset únicamente
        # Cogemos rutas aleatorias para los vídeos de dicho subset son random_state fijo para reproducibilidad
        rutas = subset['video_path'].sample(n=sample_size, random_state=42).tolist()
    
        frame_counts = []
        for ruta in rutas:
            # Con OpenCV, abrimos cada vídeo y contamos el número de frames que tiene. Si el vídeo no se puede abrir o no tiene frames, lo ignoramos.
            # ----- MELD -----------
            if df[df['video_path'] == ruta]['dataset_origin'].values[0] == 'MELD':
                cap = cv2.VideoCapture(os.path.join(path_clips_meld, ruta))
            else:
                # ----- IEMOCAP ------
                cap = cv2.VideoCapture(os.path.join(path_clips_iemocap, ruta))
            if cap.isOpened():
                frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
                if frames > 0: frame_counts.append(frames)
            cap.release()
    
        if frame_counts:
            media = np.mean(frame_counts)
            max = np.max(frame_counts)
            min = np.min(frame_counts)
            resultados.append([media,min,max])
        else:
            print("No se pudieron leer los vídeos.")
            resultados.append([0,0,0])
    return resultados


resultados_datasets = analizar_densidad_frames(df_global, MELD_CLIPS_PATH, LOCAL_DATA_ROOT)
print(f"Resultados Frames en MELD: \nMedia de frames por vídeo: {resultados_datasets[0][0]:.2f}\nMínimo: {resultados_datasets[0][1]:.2f}, Máximo: {resultados_datasets[0][2]:.2f}")
print(f"\nResultados Frames en IEMOCAP: \nMedia de frames por vídeo: {resultados_datasets[1][0]:.2f}\nMínimo: {resultados_datasets[1][1]:.2f}, Máximo: {resultados_datasets[1][2]:.2f}")
print(f"\nCon 32 frames, capturamos aprox el {32/resultados_datasets[1][0]*100:.1f}% de la información para IEMOCAP y el {32/resultados_datasets[0][0]*100:.1f}% de la información para MELD")

Resultados Frames en MELD: 
Media de frames por vídeo: 87.42
Mínimo: 2.00, Máximo: 231.00

Resultados Frames en IEMOCAP: 
Media de frames por vídeo: 123.14
Mínimo: 34.00, Máximo: 382.00

Con 32 frames, capturamos aprox el 26.0% de la información para IEMOCAP y el 36.6% de la información para MELD


Tras este análisis realizado de densidad temporal, se ha decidido establecer la resolución de muestreo a **32 frames** por vídeo. Esto permite capturar en **IEMOCAP** aproximadamente el **26%** de la información en **IEMOCAP** y un **36.6%** en **MELD** sin tener que explotar la **RAM**. Además, esta decisión permite capturar una frecuenica de muestreo de, aproximadamente, **6-8 FPS** (dependiendo del dataset), suficiente para registrar micro-expresiones faciales que ocurren en intervalos inferiores a 200 ms.

**CONCLUSIÓN**: Con **32 frames** (`num_frames = 32`), el tensor resultante tendrá dimensiones `(32,dim_model)` (dependiendo del modelo), garantizando un mayor balance entre ganularidad temporal y viabilidad de memoria para el entrenamiento de las redes recurrentes (**LSTM**). 

Definimos a continuación una función que nos extraiga los *frames* con el número indicado (en nuestro caso, **32**):

In [ ]:
def extract_frames(video_path, num_frames=32):
    """
    Abre un vídeo y extrae 'num_frames' imágenes distribuidas uniformemente.
    Devuelve: Una lista de imágenes en formato PIL.
    """
    # Abrimos el vídeo con OpenCV (en formato BGR, que es el formato nativo de OpenCV y evita gasto de CPU en convertir a RGB si no es necesario)
    cap = cv2.VideoCapture(video_path)
    
    # Verificamos si el vídeo abre
    if not cap.isOpened():
        return [] # Si no se puede abrir el vídeo, devolvemos una lista vacía

    # Leemos todos los frames del vídeo y los almacenamos en una lista:
    all_frames = []
    while True:
        ret, frame = cap.read()
        if not ret:
            break # Si no se pueden leer más frames, salimos del bucle, ya que hemos leído todos los frames disponibles
        all_frames.append(frame) # Guardamos BGR 
    
    cap.release() # Cerramos el vídeo para liberar recursos

    total_frames = len(all_frames)
    if total_frames == 0:
        return []  # Por seguridad, si el vídeo no tiene frames, devolvemos una lista vacía

    # Calculamos qué índices coger, con np.linspace creamos una secuencia espaciada uniformemente (ej: 0, 5, 10...)
    indices = np.linspace(0, total_frames - 1, num_frames, dtype=int)
    # Esto asegura que aunque el vídeo tenga menos de 'num_frames', no se generarán índices fuera del rango, y se repetirá el último frame según sea necesario
    
    # Solo convertimos a RGB y PIL los 32 frames seleccionados, para evitar gasto de CPU en convertir a RGB los frames que no vamos a usar:
    selected_frames = []
    for i in indices:
        frame_bgr = all_frames[i]
        frame_rgb = cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2RGB)
        selected_frames.append(Image.fromarray(frame_rgb))
    
    # Padding de seguridad si el video era muy corto (menos de 32 frames), repetimos el último frame hasta completar los 32:
    while len(selected_frames) < num_frames:
        selected_frames.append(selected_frames[-1])
        
    return selected_frames

---
## **ResNet50**

In [ ]:
def resnet_extractor():
    """
    Carga ResNet50 pre-entrenada y elimina la capa de clasificación (FC).
    Devuelve:
    - model: El modelo extractor listo para usar.
    - output_dim: 2048 (Número de características por frame)
    """
    
    # Cargamos los pesos entrenados en ImageNet. Esto asegura que la red sepa ya reconocer formas, luces y texturas (sesgo inductivo de la CNN):
    weights = models.ResNet50_Weights.DEFAULT
    model = models.resnet50(weights=weights)
    
    # ----- ELIMINAMOS ÚLTIMA CAPA DE CLASIFICACIÓN -----
    # La arquitectura ResNet original termina en una capa 'fc' (Linear) que clasifica en 1000 clases.
    # Nosotros queremos el vector justo antes de esa clasificación
    modules = list(model.children())[:-1] # Con list(model.children())[:-1] cogemos todas las capas menos la última
    model = nn.Sequential(*modules) # Creamos un nuevo modelo con todas las capas excepto la última
    
    # Congelamos los pesos (Freeze), ya que no queremos entrenar la CNN, solo usarla directamente:
    for param in model.parameters():
        param.requires_grad = False
        
    model.to(device)
    model.eval() # Ponemos en modo evaluación 
    
    return model, 2048  # ResNet50 siempre devuelve vectores de tamaño 2048

# Definimos también las transformaciones que ResNet (al igual que EfficientNet) necesita para funcionar:
# Son reglas estrictas de ImageNet: normalización con media/std específicos
preprocess = transforms.Compose([ 
    transforms.Resize(256), # Redimensionamos a 256x256
    transforms.CenterCrop(224), # Recortamos al centro para quedarnos con 224x224, que es el tamaño que ResNet y EfficientNet esperan
    transforms.ToTensor(), # Convertimos a tensor (esto también escala los píxeles a [0,1])
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]), # Normalización específica de ImageNet
])

# Con esto, ya tenemos lista la configuración y preprocesamiento de los frames para ResNet

-------
## **EfficientNet-B0**

In [ ]:
def efficientnet_extractor():
    """
    Carga EfficientNet-B0 y adapta para extracción de características.
    Devuelve:
    - model: El extractor.
    - output_dim: 1280
    """
    
    #  Cargamos los pesos
    weights = models.EfficientNet_B0_Weights.DEFAULT
    model = models.efficientnet_b0(weights=weights)
    
    # -----ELIMINAMOS ÚLTIMA CAPA DE CLASIFICACIÓN -----
    # En EfficientNet, la capa de clasificación se llama 'classifier'
    # Únicamente queremos lo que se obtiene de las 'features' y pasa por el 'avgpool':
    model = nn.Sequential(
        model.features,  #Seleecionamos las features 
        model.avgpool  # y avgpool
    )
    
    # Congelamos los pesos:
    for param in model.parameters():
        param.requires_grad = False
        
    model.to(device)
    model.eval()
    
    return model, 1280  # EfficientNet-B0 devuelve vectores de 1280

# Se definen las mismas transformaciones que para ResNet (preprocess), ya que es el estándar en ImageNet.
# Con esto ya tenemos configurado EfficientNet-B0

-------
## **ViT (Vision Transformer)**

In [ ]:
def vit_extractor():
    """
    Carga ViT-B/16 pre-entrenado y adapta para extracción de características.
    Devuelve:
    - model: El extractor.
    - output_dim: 768
    """
    
    # Cargamos los pesos pre-entrenados de ViT-B/16
    weights = models.ViT_B_16_Weights.DEFAULT
    model = models.vit_b_16(weights=weights)
    # -----ELIMINAMOS ÚLTIMA CAPA DE CLASIFICACIÓN -----
    # En ViT, la capa de clasificación se llama 'head'. Para extraer las características directamente, 
    # podemos reemplazar esta capa por una identidad, lo que hará que el modelo devuelva directamente el vector de características (el CLS token)
    model.head = nn.Identity()
    
    # Congelamos los pesos:
    for param in model.parameters():
        param.requires_grad = False
        
    model.to(device)
    model.eval()
    
    return model, 768  # ViT-B/16 devuelve vectores de 768

# Las transformaciones son las mismas que para ResNet y EfficientNet
# Con esto ya tenemos configurado ViT-B/16

----
### ***Feed-Forward*. Extracción de características.**

Una vez ya tenemos definido y cargados los modelos, extraemos los embeddings de cada uno de los vídeos (y los almacenamos en los directorios creados):

**NOTA**: Las anteriores celdas de código muestran, a modo de ejemplo, la carga de cada uno de los modelos de forma más detallada, con motivo principalmente demostrativo. A continuación, se presenta la celda de código exacta del script `.py` ejecutado en el Servidor DGX. Los *embeddings* se obtienen inicialmente en este servidor, y luego se empaquetan (`.tar.gz`) y se envían para almacernalos en **Drive**.

In [ ]:
# ===========================================================================
#  BLOQUE DE EJECUCIÓN EN SERVIDOR DGX 
# ===========================================================================
# Debido a la carga computacional de procesar el dataset completo con tres
# arquitecturas (especialmente Vision Transformers), este proceso se ha ejecutado
# en el servidor NVIDIA DGX Spark (cyberdatalab.um.es) mediante un script .py optimizado.
#
# A continuación se muestra el código exacto utilizado en dicho script para la
# generación de los embeddings. Esta celda se mantiene como bloque demostrativo.
# ======================================================================================

# Carga previa de todas las librerías y paquetes necesarios para el script:
import os
import cv2
import torch
import argparse
import numpy as np
import pandas as pd
import torch.nn as nn
from PIL import Image
from tqdm import tqdm
from torchvision import models, transforms
from torch.utils.data import Dataset, DataLoader
    

# ------------------------------------------------------------------------
# 1. Definición del Dataset para Carga Paralela (Multiprocessing)
# ------------------------------------------------------------------------
class VideoDataset(Dataset):
    def __init__(self, df, root_meld, root_iemocap, transform=None):
        self.df = df
        self.root_meld = root_meld
        self.root_iemocap = root_iemocap
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        # Selección de ruta según el origen del vídeo
        if row['dataset_origin'] == 'MELD':
            video_path = os.path.join(self.root_meld, row['video_path'])
        else:
            video_path = os.path.join(self.root_iemocap, row['video_path'])
        
        # Extracción de 32 frames uniformes del vídeo
        frames = self.extract_frames(video_path)
        if self.transform and len(frames) > 0:
            frames = torch.stack([self.transform(f) for f in frames])
        
        # Retornamos el tensor de frames y el nombre formateado para el archivo .npy
        file_name = f"{row['Dialogue_ID']}_{row['Utterance_ID']}.npy".replace("/", "_")
        return frames, file_name

    def extract_frames(self, video_path, num_frames=32):
        cap = cv2.VideoCapture(video_path)
        all_frames = []
        while True:
            ret, frame = cap.read()
            if not ret: break
            all_frames.append(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        cap.release()
        
        if not all_frames: return []
        indices = np.linspace(0, len(all_frames) - 1, num_frames, dtype=int)
        return [Image.fromarray(all_frames[i]) for i in indices]

# -------------------------------------------------------------------------
# 2. Lógica Principal de Extracción
# -------------------------------------------------------------------------
def main_extraction(model_name, num_workers=8):
    """
    Función principal de extracción diseñada para ejecución paralela en HPC.
    Args:
    - model_name (str): Nombre del modelo a usar ('resnet', 'efficientnet' o 'vit').
    - num_workers (int): Número de procesos paralelos para DataLoader.
    Devuelve:
    - Guarda los embeddings extraídos en archivos .npy en el directorio correspondiente.
    """
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    output_dir = f"./features_{model_name}"
    os.makedirs(output_dir, exist_ok=True)

    # 1. Configuración del Extractor según Arquitectura
    if model_name == 'resnet':
        m = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)
        model = nn.Sequential(*list(m.children())[:-1])  # Eliminamos la capa de clasificación
    elif model_name == 'efficientnet':
        m = models.efficientnet_b0(weights=models.EfficientNet_B0_Weights.DEFAULT)
        model = nn.Sequential(m.features, m.avgpool) # Solo queremos las features y avgpool, no la clasificación
    elif model_name == 'vit':
        # Instanciamos el modelo base
        model = models.vit_b_16(weights=models.ViT_B_16_Weights.DEFAULT)
        # Reemplazamos la caabeza (la capa que clasifica en 1000 clases) por una identidad para obtener las features directamente
        # Esto hace que el modelo devuelva directamente el vector de características (el CLS token)
        model.head = nn.Identity()
    
    for p in model.parameters(): 
        p.requires_grad = False
    model.to(device).eval()

    # 2. Pipeline de Preprocesamiento e Inferencia
    preprocess = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])

    # 3. Ejecución Paralela (DataLoader con Workers)
    df = pd.read_csv("Multimodal_Stress_Dataset.csv")
    dataset = VideoDataset(df, "./data/MELD_CLIPS", "./data", transform=preprocess)
    dataloader = DataLoader(dataset, batch_size=1, num_workers=num_workers)

    with torch.no_grad():
        for frames, name in tqdm(dataloader, desc=f"Extrayendo {model_name}"):
            if frames.shape[1] == 0: 
                continue
            
            frames = frames.squeeze(0).to(device) # Shape: (32, 3, 224, 224)
            embeddings = model(frames)
            # Embeddings shape tras pasar por los modelos:
            # ResNet/EfficientNet -> (32, Dim, 1, 1) -> Necesitamos aplicar flatten
            # ViT (con Wrapper) -> (32, 768)  -> Ya está plano
            
            if model_name != 'vit':
                embeddings = embeddings.flatten(start_dim=1)
            
            np.save(os.path.join(output_dir, name[0]), embeddings.cpu().numpy())
    

# -------------------------------------------------------------------------
# PUNTO DE ENTRADA DEL SCRIPT (Main)
# -------------------------------------------------------------------------
if __name__ == '__main__':
    # En el script real (.py), el modelo se recibe como argumento de consola, al igual que el número de workers para la ejecución paralela:
    # >>> python feature_extraction.py --model vit --workers 16
    
    # parser = argparse.ArgumentParser()
    # parser.add_argument('--model', type=str, required=True)
    # parser.add_argument('--workers', type=int, default=8)
    # args = parser.parse_args()
    # main_extraction(args.model, num_workers=args.workers)

    pass # En este bloque no se ejecuta nada, ya que la extracción se ha realizado en el servidor DGX mediante el script.

#### **EXTRACCIÓN EMBEDDINGS RESULTANTES DESDE DRIVE (`.tar.gz`)**


In [8]:
tar_files = {
    TAR_PATH_RESNET: OUTPUT_DIR_RESNET,
    TAR_PATH_EFFICIENTNET: OUTPUT_DIR_EFFICIENTNET,
    TAR_PATH_VIT: OUTPUT_DIR_VIT
}

for tar_path, extract_dir in tar_files.items():
    if not os.path.exists(extract_dir) or len(os.listdir(extract_dir)) == 0:
        print(f"Descomprimiendo {os.path.basename(tar_path)} en {extract_dir}...")
        os.makedirs(extract_dir, exist_ok=True)
        with tarfile.open(tar_path, 'r:gz') as tar_ref: #'r:gz': lectura de archivo gzip comprimido
            tar_ref.extractall(path=extract_dir)
    else:
        print(f"Los embeddings ya existen en local: {extract_dir}. ")


Descomprimiendo features_resnet.tar.gz en /content/data/EMBEDDINGS_VISUAL_RESNET...


/tmp/ipython-input-2851875848.py:12: DeprecationWarning: Python 3.14 will, by default, filter extracted tar archives and reject files or modify their metadata. Use the filter argument to control this behavior.
  tar_ref.extractall(path=extract_dir)


Descomprimiendo features_efficientnet.tar.gz en /content/data/EMBEDDINGS_VISUAL_EFFICIENTNET...
Descomprimiendo features_vit.tar.gz en /content/data/EMBEDDINGS_VISUAL_ViT...


### **Sanity Check**. 

Realizamos a continuación una verficación rápida de que todos los *embeddings* se han obtenido correctamente (en total y por cada corpus/partición), además de que se han guardado correctamente en el cloud (**Google Drive**).

Este *Sanity Check* realiza lo siguiente:
* Comprueba el número de archivos guardados (debe ser igual a los vídeos indicados en el CSV).
* Muestra un ejemplo de archivo guardado por cada modelo (**ResNet**, **EfficientNet** y **ViT**) para verificar que se han guardado correctamente.
* Se muestran mensajes de error claros en caso de un guardado incorrecto, indicando el número de archivos **esperados** en relación a los **encontrados** (obtenidos).
* Muestra el número de embeddings obtenidos por cada *dataset* (**MELD** e **IEMOCAP**), además de por cada *split*/partición (`train`, `test`, `dev`) para verificar que se han guardado correctamente los archivos de *embeddings* de ambos *datasets* y de cada *split*.

In [ ]:
# ----------- SANITY CHECK: VALIDACIÓN DE EMBEDDINGS GUARDADOS ---------

output_dirs = {
    'resnet': os.path.join(OUTPUT_DIR_RESNET, 'features_resnet'),
    'efficientnet': os.path.join(OUTPUT_DIR_EFFICIENTNET, 'features_efficientnet'),
    'vit': os.path.join(OUTPUT_DIR_VIT, 'features_vit')
}

# 1. CONTEO GENERAL DE ARCHIVOS GENERADOS
print("\nVALIDACIÓN DE EMBEDDINGS GUARDADOS:")
total_videos_expected = len(df_global)
resnet_files = [f for f in os.listdir(output_dirs['resnet']) if f.endswith('.npy')]
efficientnet_files = [f for f in os.listdir(output_dirs['efficientnet']) if f.endswith('.npy')]
vit_files = [f for f in os.listdir(output_dirs['vit']) if f.endswith('.npy')]

print(f"Vídeos esperados (CSV): {total_videos_expected}")
print(f"Embeddings ResNet guardados: {len(resnet_files)}")
print(f"Embeddings EfficientNet guardados: {len(efficientnet_files)}")
print(f"Embeddings ViT guardados: {len(vit_files)}")

# 2. VERIFICACIÓN POR DATASET (MELD vs IEMOCAP)
print("\nVALIDACIÓN POR DATASET:")
for dataset in ['MELD', 'IEMOCAP']:
    dataset_videos = df_global[df_global['dataset_origin'] == dataset]
    expected_count = len(dataset_videos)

    # Contamos archivos que corresponden a este dataset en concreto
    dataset_resnet_files = []
    dataset_efficientnet_files = []
    dataset_vit_files = []

    for _, row in dataset_videos.iterrows():
        filename = f"{row['Dialogue_ID']}_{row['Utterance_ID']}.npy".replace("/","_")

        if filename in resnet_files:
            dataset_resnet_files.append(filename)
        if filename in efficientnet_files:
            dataset_efficientnet_files.append(filename)
        if filename in vit_files:
            dataset_vit_files.append(filename)

    print(f"\n{dataset}:")
    print(f"Vídeos esperados: {expected_count}")
    print(f"ResNet embeddings: {len(dataset_resnet_files)} ({len(dataset_resnet_files)/expected_count*100:.1f}%)")
    print(f"EfficientNet embeddings: {len(dataset_efficientnet_files)} ({len(dataset_efficientnet_files)/expected_count*100:.1f}%)")
    print(f"ViT embeddings: {len(dataset_vit_files)} ({len(dataset_vit_files)/expected_count*100:.1f}%)")

    try:
        assert len(dataset_resnet_files) == expected_count, f"FALLO en {dataset}: ResNet esperaba {expected_count}, encontró {len(dataset_resnet_files)}"
        assert len(dataset_efficientnet_files) == expected_count, f"FALLO en {dataset}: EfficientNet esperaba {expected_count}, encontró {len(dataset_efficientnet_files)}"
        assert len(dataset_vit_files) == expected_count, f"FALLO en {dataset}: ViT esperaba {expected_count}, encontró {len(dataset_vit_files)}"
    except AssertionError as e:
        print(f"{dataset}: {str(e)}")

# 3. VERIFICACIÓN POR SPLIT/PARTICIÓN
print("\nVALIDACIÓN POR SPLIT/PARTICIÓN:")
for split in ['train', 'dev', 'test']:
    split_videos = df_global[df_global['split'] == split]
    expected_count = len(split_videos)

    if expected_count == 0:
        print(f"{split.upper()}: No hay vídeos en esta partición")
        continue

    # Contamos archivos que corresponden a este split
    split_resnet_files = []
    split_efficientnet_files = []
    split_vit_files = []

    for _, row in split_videos.iterrows():
        filename = f"{row['Dialogue_ID']}_{row['Utterance_ID']}.npy".replace("/","_")

        if filename in resnet_files:
            split_resnet_files.append(filename)
        if filename in efficientnet_files:
            split_efficientnet_files.append(filename)
        if filename in vit_files:
            split_vit_files.append(filename)

    print(f"\n{split.upper()}:")
    print(f"Vídeos esperados: {expected_count}")
    print(f"ResNet embeddings: {len(split_resnet_files)} ({len(split_resnet_files)/expected_count*100:.1f}%)")
    print(f"EfficientNet embeddings: {len(split_efficientnet_files)} ({len(split_efficientnet_files)/expected_count*100:.1f}%)")
    print(f"ViT embeddings: {len(split_vit_files)} ({len(split_vit_files)/expected_count*100:.1f}%)")

    # Assert por split:
    try:
        assert len(split_resnet_files) == expected_count, f"FALLO en {split}: ResNet esperaba {expected_count}, encontró {len(split_resnet_files)}"
        assert len(split_efficientnet_files) == expected_count, f"FALLO en {split}: EfficientNet esperaba {expected_count}, encontró {len(split_efficientnet_files)}"
        assert len(split_vit_files) == expected_count, f"FALLO en {split}: ViT esperaba {expected_count}, encontró {len(split_vit_files)}"
    except AssertionError as e:
        print(f"{split.upper()}: {str(e)}")

# 4. MOSTRAMOS EJEMPLOS DE EMBEDDINGS GUARDADOS
print("\nEJEMPLOS DE EMBEDDINGS GUARDADOS:")
if len(resnet_files) > 0:
    example_resnet = resnet_files[0]
    resnet_path = os.path.join(output_dirs['resnet'], example_resnet)
    resnet_data = np.load(resnet_path)
    print(f"\nResNet ejemplo: {example_resnet}")
    print(f"Shape: {resnet_data.shape}")
    print(f"Tamaño: {resnet_data.nbytes/1024:.1f} KB")
    print(f"Tipo: {resnet_data.dtype}")

if len(efficientnet_files) > 0:
    example_eff = efficientnet_files[0]
    eff_path = os.path.join(output_dirs['efficientnet'], example_eff)
    eff_data = np.load(eff_path)
    print(f"\nEfficientNet ejemplo: {example_eff}")
    print(f"Shape: {eff_data.shape}")
    print(f"Tamaño: {eff_data.nbytes/1024:.1f} KB")
    print(f"Tipo: {eff_data.dtype}")

if len(vit_files) > 0:
    example_vit = vit_files[0]
    vit_path = os.path.join(output_dirs['vit'], example_vit)
    vit_data = np.load(vit_path)
    print(f"\nViT ejemplo: {example_vit}")
    print(f"Shape: {vit_data.shape}")
    print(f"Tamaño: {vit_data.nbytes/1024:.1f} KB")
    print(f"Tipo: {vit_data.dtype}")


VALIDACIÓN DE EMBEDDINGS GUARDADOS:
Vídeos esperados (CSV): 21219
Embeddings ResNet guardados: 21219
Embeddings EfficientNet guardados: 21219
Embeddings ViT guardados: 21219

VALIDACIÓN POR DATASET:

MELD:
Vídeos esperados: 13704
ResNet embeddings: 13704 (100.0%)
EfficientNet embeddings: 13704 (100.0%)
ViT embeddings: 13704 (100.0%)

IEMOCAP:
Vídeos esperados: 7515
ResNet embeddings: 7515 (100.0%)
EfficientNet embeddings: 7515 (100.0%)
ViT embeddings: 7515 (100.0%)

VALIDACIÓN POR SPLIT/PARTICIÓN:

TRAIN:
Vídeos esperados: 14318
ResNet embeddings: 14318 (100.0%)
EfficientNet embeddings: 14318 (100.0%)
ViT embeddings: 14318 (100.0%)

DEV:
Vídeos esperados: 2644
ResNet embeddings: 2644 (100.0%)
EfficientNet embeddings: 2644 (100.0%)
ViT embeddings: 2644 (100.0%)

TEST:
Vídeos esperados: 4257
ResNet embeddings: 4257 (100.0%)
EfficientNet embeddings: 4257 (100.0%)
ViT embeddings: 4257 (100.0%)

EJEMPLOS DE EMBEDDINGS GUARDADOS:

ResNet ejemplo: 12_dev_dia12_utt2.npy
Shape: (32, 2048)
Tama

Se confirma que se han generado correctamente todos los *embeddings* **visuales** (este paso es imprescindible para construir la base de nuestros modelos posteriores, ya que estas serán las entradas visuales que recibirán inicialmente). Por tanto, contamos con **21219** embeddings visuales obtenidos con **ResNet50** de tamaño `(32,2048)`, otros **21219** obtenidos con **EfficientNet** de tamaño `(32,1280)`, y **21219** obtenidos con **ViT** de tamaño `(32,768)`.